# API KEY

In [1]:
# 파일을 읽기 모드로 열기
with open("C:\\Users\\lee\\Desktop\\bybit\\api.txt", "r") as file:
    api_list = []
    # 파일의 각 줄에 대해 반복
    for line in file:
        # 각 줄 출력
        api_list.append(line.strip())  # strip()은 줄 바꿈 문자(\n) 등을 제거하여 출력합니다.

# TRADE & TP/SL

In [ ]:
from pybit.unified_trading import HTTP

key = api_list[0]
secret = api_list[1]

session = HTTP(
    # testnet=True,
    api_key=key,
    api_secret=secret,
)

# 교차, 격리 / 레버리지 배율
print(session.switch_margin_mode(
    category="linear",
    symbol="MNTUSDT",
    tradeMode=1, # 0: cross margin. 1: isolated margin
    buyLeverage="1",
    sellLeverage="1",
))

# 주문 조건
print(session.place_order(
    category="linear",
    symbol="MNTUSDT",
    side="Buy",
    orderType="Market",
    qty="1",
    # price="15600", 지정가일 때만 필요
    timeInForce="IOC",
    orderLinkId="one-view",
    isLeverage=1,
    orderFilter="Order",
    
    # 추가
    takeProfit = 0.5,
    stopLoss = 0.42,
))